### Music Artist Recommender Engine built with Apache Spark and Python

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pyspark.sql.functions import concat, col, lit
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit, CrossValidator

#### Load data
Load the two datasets of music artist information and user ratings.

In [4]:
artist = spark.read.load("/FileStore/tables/artist.csv", format='csv', header = True)
music = spark.read.load("/FileStore/tables/music.csv", format='csv', header = True)

In [5]:
artist = artist.select("artistID", "artist")
artist.show(10)

+--------+--------------+
artistID| artist|
+--------+--------------+
 1000001| "Bobby ""O"""|
 1000002| "Jimmy ""Z"""|
 1000003| '68 Comeback|
 1000004| 'Til Tuesday|
 1000005|The (EC) Nudes|
 1000006| .38 Special|
 1000008| 1 + 1|
 1000009|1 Of The Girls|
 1000010| 1,000 Clowns|
 1000011| 10 K.A.N.'s|
+--------+--------------+
only showing top 10 rows

In [6]:
music = music.select("userID", "artistID", "rating")
music.show(10)

+------+--------+------+
userID|artistID|rating|
+------+--------+------+
 1| 1000125| 90|
 1| 1006373| 100|
 1| 1006978| 90|
 1| 1007035| 100|
 1| 1007098| 100|
 1| 1007723| 100|
 1| 1008659| 100|
 1| 1008916| 100|
 1| 1012809| 70|
 1| 1014635| 100|
+------+--------+------+
only showing top 10 rows

#### Data processing

In [8]:
# Reset userID, artistID and rating dtypes
artist = artist.withColumn('artistID', artist['artistID'].cast('int'))
music = (music.withColumn('userID', music['userID'].cast('int'))
           .withColumn('artistID', music['artistID'].cast('int'))
           .withColumn('rating', music['rating'].cast('float')))

In [9]:
music.printSchema()

root
-- userID: integer (nullable = true)
-- artistID: integer (nullable = true)
-- rating: float (nullable = true)

In [10]:
artist.printSchema()

root
-- artistID: integer (nullable = true)
-- artist: string (nullable = true)

In [11]:
# Rating should be from 0-100, 255 implies do not play anymore
music = music.filter(music['rating'] <= 100)

In [12]:
music.describe().show()

+-------+------------------+------------------+-----------------+
summary| userID| artistID| rating|
+-------+------------------+------------------+-----------------+
 count| 997910| 997910| 997910|
 mean| 8413.697037809021|1033732.6757513202|53.03472357226604|
 stddev|4857.5949623641845| 32930.94190008406|39.97780729890414|
 min| 1| 24538| 0.0|
 max| 16685| 1101719| 100.0|
+-------+------------------+------------------+-----------------+

#### Data Exploration
Check the number of artists and users.
Check the artist that with most ratings and users made most ratings.

In [14]:
top_artist = music.groupBy("artistID").count().orderBy('count', ascending=False)
top_user = music.groupBy("userID").count().orderBy('count', ascending=False)
top_artist.createOrReplaceTempView("top_artist")
top_user.createOrReplaceTempView("top_user")
# artist.createOrReplaceTempView("artist")

In [15]:
pop_artist = spark.sql("select * \
                       from top_artist join artist \
                       on top_artist.artistID=artist.artistID \
                       order by top_artist.count desc")

In [16]:
# Show top 20 artist that have most ratings.
pop_artist.show(20)

+--------+-----+--------+--------------------+
artistID|count|artistID| artist|
+--------+-----+--------+--------------------+
 1053507| 6788| 1053507| Linkin Park|
 1098798| 5282| 1098798| Evanescence|
 1022226| 4970| 1022226|Red Hot Chili Pep...|
 1030879| 4839| 1030879| Missy Elliott|
 1037847| 4636| 1037847| Eminem|
 1099097| 4448| 1099097| Chingy|
 1042768| 4335| 1042768| Staind|
 1010787| 4298| 1010787| Green Day|
 1017240| 4194| 1017240| Matchbox Twenty|
 1017874| 4181| 1017874| Metallica|
 1045704| 4128| 1045704| Nelly|
 1030760| 3991| 1030760| Creed|
 1016600| 3842| 1016600| Madonna|
 1053653| 3737| 1053653| Coldplay|
 1044487| 3685| 1044487| Sean Paul|
 1051565| 3663| 1051565| Beyonce|
 1014312| 3657| 1014312| Lil' Kim|
 1043454| 3526| 1043454| 50 Cent|
 1030666| 3522| 1030666| Blink 182|
 1008023| 3510| 1008023| Eagles|
+--------+-----+--------+--------------------+
only showing top 20 rows

In [17]:
# top 20 users that have made most ratings.
top_user.show(20)

+------+-----+
userID|count|
+------+-----+
 12466| 8011|
 9877| 3627|
 2663| 3123|
 12213| 3056|
 10226| 2873|
 3950| 2813|
 11520| 2735|
 4395| 2381|
 14963| 2312|
 2500| 2090|
 3414| 2058|
 7266| 2053|
 3176| 1910|
 12993| 1826|
 15105| 1814|
 143| 1767|
 11189| 1700|
 9286| 1680|
 2714| 1632|
 2519| 1583|
+------+-----+
only showing top 20 rows

In [18]:
temp1 = sum(music.groupBy("artistID").count().toPandas()['count'] == 1)
temp2 = music.select('artistID').distinct().count()
print ('{} out of {} artist with one rating'.format(temp1, temp2))
temp3 = sum(music.groupBy("userID").count().toPandas()['count'] == 1)
temp4 = music.select('userID').distinct().count()
print ('{} out of {} user rate one artist'.format(temp3, temp4))

3921 out of 13925 artist with one rating
1667 out of 16674 user rate one artist

#### Split data into train and test.

In [20]:
# The datasets will be repeatedly used, persist them in memory using the cache function
train, test = music.randomSplit([0.8, 0.2], seed=1234)
train.cache()
test.cache()

Out[ 15 ]: DataFrame[userID: int, artistID: int, rating: float]

#### Build the recommender model using ALS

In [22]:
# If the rating matrix is derived from another source of information, you can set implicitPrefs to True to get better results.
als = ALS(userCol='userID', itemCol='artistID', ratingCol='rating', coldStartStrategy='drop')

In [23]:
# A CrossValidator requires an Estimator, a set of Estimator ParamMaps, and an Evaluator.
# We use a ParamGridBuilder to construct a grid of parameters to search over.
# this grid will have 2 x 5 x 4 = 40 parameter settings for CrossValidator to choose from.
paramGrid = ParamGridBuilder()\
    .addGrid(als.maxIter, [5, 10]) \
    .addGrid(als.regParam, [0.05, 0.1, 0.2, 0.4, 0.8])\
    .addGrid(als.rank, [6, 8, 10, 12])\
    .build()

In [24]:
crossval = CrossValidator(estimator=als,
                          estimatorParamMaps=paramGrid,
                          evaluator=RegressionEvaluator(metricName="rmse", labelCol="rating",
                          predictionCol="prediction"),
                          numFolds=2)

In [25]:
# Run cross-validation, and choose the best set of parameters.
cvModel = crossval.fit(train)

In [26]:
bestModel = cvModel.bestModel

In [27]:
# Make predictions on test data. model is the model with combination of parameters that performed best.
predictions = bestModel.transform(test)
predictions.show(5)

+------+--------+------+----------+
userID|artistID|rating|prediction|
+------+--------+------+----------+
 10587| 1001043| 0.0| -6.029426|
 9001| 1001043| 90.0| 65.9859|
 489| 1001043| 50.0| 54.676533|
 15484| 1001043| 90.0| 60.15025|
 13244| 1001043| 0.0| 4.819909|
+------+--------+------+----------+
only showing top 5 rows

In [28]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse_test = evaluator.evaluate(predictions)

In [29]:
# RMSE of the testing dataset
rmse_test

Out[ 23 ]: 25.213418384778027

Compare the RMSE of ALS model with the benchmark results using mean rating of each artist as the prediction.

In [31]:
mean_df = train.groupby('artistId').mean().select('artistID','avg(rating)')
mean_df = mean_df.withColumnRenamed('avg(rating)','prediction')
mean_df.show(5)

+--------+------------------+
artistID| prediction|
+--------+------------------+
 1030811| 47.25454545454546|
 1053407| 72.62404942965779|
 1030666| 68.54035714285715|
 1053349| 34.61802575107296|
 1098379|20.571428571428573|
+--------+------------------+
only showing top 5 rows

In [32]:
test.createOrReplaceTempView("test")
mean_df.createOrReplaceTempView("mean_df")
mean_sql = spark.sql("select test.*, mean_df.prediction \
                       from test join mean_df \
                       on test.artistID = mean_df.artistID")

In [33]:
# RMSE of mean ratings
rmse_mean = evaluator.evaluate(mean_sql)
rmse_mean

Out[ 26 ]: 36.82790508023549

ALS model has RMSE much less than using mean rating as prediction.

#### Make recommendations for a selected user

In [36]:
# Generate top 10 user recommendations for a specified set of movies
users = music.select(als.getUserCol()).distinct().limit(1)
userSubsetRecs = bestModel.recommendForUserSubset(users, 10)
users.show()

+------+
userID|
+------+
 296|
+------+

In [37]:
userSubsetRecs = userSubsetRecs.toPandas()
ans = []
for i in range(10):
  ans.append(userSubsetRecs[['recommendations']].iloc[0,0][i][0])
artist.where(col("artistID").isin(ans)).show()

+--------+--------------------+
artistID| artist|
+--------+--------------------+
 1000209| The 7A3|
 1009025| Karel Fialka|
 1011370| Bo Hansson|
 1013401|Jive Bunny & The ...|
 1017294| Maurice Jarre|
 1021365| Popol Vuh|
 1027393| Tramp|
 1035531| M/A/R/R/S|
 1043893| Richard O'Brien|
 1097379| Mark Shreeve|
+--------+--------------------+